In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [11]:
torch.manual_seed(42)

In [12]:
device='cuda'

In [13]:
df = pd.read_csv('../../3OEC_current_flow.csv')

df["O2_avg"] = df[["O2_S1", "O2_S2", "O2_S3"]].mean(axis=1)

In [14]:
def MARE_np(prediction, truth):
    epsilon = 1e-8  # Small value to prevent division by zero
    return np.sum((np.abs(prediction - truth)) / np.abs(truth + epsilon)) / len(truth)

In [15]:
from datetime import datetime, timedelta

start_time_11 = datetime(2017, 7, 11, 14, 0, 0)
end_time_11 = datetime(2017, 7, 12, 8, 0, 0)

start_time_13 = datetime(2017, 7, 13, 11, 0, 0)
end_time_13 = datetime(2017, 7, 14, 6, 0, 0)

start_time_15 = datetime(2017, 7, 15, 10, 0, 0)
end_time_15 = datetime(2017, 7, 16, 6, 0, 0)

start_time_16 = datetime(2017, 7, 16, 16, 0, 0)
end_time_16 = datetime(2017, 7, 17, 6, 0, 0)

deployments = {
    "3oec_2017_7_11_12": {"start": start_time_11, "end": end_time_11},
    "3oec_2017_7_13_14": {"start": start_time_13, "end": end_time_13},
    "3oec_2017_7_15_16": {"start": start_time_15, "end": end_time_15},
    "3oec_2017_7_16_17": {"start": start_time_16, "end": end_time_16}
}

date_ranges = []

for deployment_name, deployment_info in deployments.items():
    start_time = deployment_info["start"]
    end_time = deployment_info["end"]
    if deployment_name == "3oec_2017_7_13_14":
        start_time -= timedelta(seconds=0.125)
    print(start_time)

    # Calculate total seconds and number of measurements
    total_seconds = (end_time - start_time).total_seconds() + 0.125
    num_measurements = int(total_seconds * 8)

    # Create DatetimeIndex for the deployment
    date_range = pd.date_range(start=start_time, periods=num_measurements, freq=f'{1000/8}ms')
    print(date_range[0], date_range[-1])
    print(len(date_range))
    date_ranges.append(pd.Series(date_range))

# Concatenate all DatetimeIndexes
complete_index = pd.concat(date_ranges)

# Set the complete index to your DataFrame
df.index = complete_index

2017-07-11 14:00:00
2017-07-11 14:00:00 2017-07-12 08:00:00
518401
2017-07-13 10:59:59.875000
2017-07-13 10:59:59.875000 2017-07-14 06:00:00
547202
2017-07-15 10:00:00
2017-07-15 10:00:00 2017-07-16 06:00:00
576001
2017-07-16 16:00:00
2017-07-16 16:00:00 2017-07-17 06:00:00
403201


In [16]:
# drop deployment column and resample
df_resampled = df.drop(columns=['deployment', 't', 't_increase', 'Vx', 'Vy', 'Vz', 'P', 'O2_S1', 'O2_S2', 'O2_S3']).resample('1min').mean()

In [17]:
first_piece = df_resampled["2017-07-11":"2017-07-12 06:00:00"]
second_piece = df_resampled["2017-07-13 12:00:00":"2017-07-14 06:00:00"]
third_piece = df_resampled["2017-07-15 12:00:00":"2017-07-16 6:00:00"]
fourth_piece = df_resampled["2017-07-16 16:00:00":"2017-07-17"]

In [18]:
def create_sequences(data, seq_length):
    """
    Creates sequences and their corresponding target sequences from the input data.
    The target sequence is half the size of the training sequence.

    Args:
        data (array-like): The input data.
        seq_length (int): The length of each training sequence.

    Returns:
        np.ndarray: Array of input sequences.
        np.ndarray: Array of target sequences (half the length of input sequences).
    """
    target_length = seq_length // 3  # Target is 1/3 the size of the training sequence

    if len(data) < seq_length + target_length:
        raise ValueError("Data length must be at least seq_length + target_length.")

    sequences = []
    targets = []
    
    for i in range(len(data) - seq_length - target_length + 1):
        sequences.append(data[i:i+seq_length])                        # Input sequence
        targets.append(data[i+seq_length:i+seq_length+target_length]) # Target sequence

    return np.array(sequences), np.array(targets)


In [19]:
# data piece 1

column_indices = {name: i for i, name in enumerate(first_piece.columns)}
n = len(first_piece)
train_df1 =first_piece

# Normalize the data (each partition separately)
train_mean1 = train_df1.mean()
train_std1 = train_df1.std()

train_df1 = (train_df1 - train_mean1) / train_std1

# make sequences
train_seq1, train_labels1 = create_sequences(train_df1.values, 90)


# make tensors
train_seq_tensor1 = torch.FloatTensor(train_seq1).to(device=device)  # (batch, seq_length, input_dim)
train_label_tensor1 = torch.FloatTensor(train_labels1).to(device=device) # (batch, output_dim)

# data piece 2
column_indices = {name: i for i, name in enumerate(first_piece.columns)}
n = len(second_piece)
train_df2 =second_piece

# Normalize the data (each partition separately)
train_mean2 = train_df2.mean()
train_std2 = train_df2.std()

train_df2 = (train_df2 - train_mean2) / train_std2


# make sequences
train_seq2, train_labels2 = create_sequences(train_df2.values, 90)


# make tensors
train_seq_tensor2 = torch.FloatTensor(train_seq2).to(device=device)  # (batch, seq_length, input_dim)
train_label_tensor2 = torch.FloatTensor(train_labels2).to(device=device) # (batch, output_dim)

# data piece 3
column_indices = {name: i for i, name in enumerate(first_piece.columns)}
n = len(third_piece)
train_df3 = third_piece


# Normalize the data (each partition separately)
train_mean3 = train_df3.mean()
train_std3 = train_df3.std()

train_df3 = (train_df3 - train_mean3) / train_std3

# make sequences
train_seq3, train_labels3 = create_sequences(train_df3.values, 90)

# make tensors
train_seq_tensor3 = torch.FloatTensor(train_seq3).to(device=device)  # (batch, seq_length, input_dim)
train_label_tensor3 = torch.FloatTensor(train_labels3).to(device=device) # (batch, output_dim)

# data piece 4
column_indices = {name: i for i, name in enumerate(first_piece.columns)}
n = len(fourth_piece)
train_df4 = fourth_piece

# Normalize the data (each partition separately)
train_mean4 = train_df4.mean()
train_std4 = train_df4.std()

train_df4 = (train_df4 - train_mean4) / train_std4

# make sequences
train_seq4, train_labels4 = create_sequences(train_df4.values, 90)

# make tensors
train_seq_tensor4 = torch.FloatTensor(train_seq4).to(device=device)  # (batch, seq_length, input_dim)
train_label_tensor4 = torch.FloatTensor(train_labels4).to(device=device) # (batch, output_dim)


In [20]:
normalized_test_1 = (first_piece[:] - first_piece.mean())/first_piece.std()
test_seq_1, test_labels_1 = create_sequences(normalized_test_1.values, 90)
test_tensor_seq_1 = torch.FloatTensor(test_seq_1).to(device=device)
test_tensor_labels_1 = torch.FloatTensor(test_labels_1).to(device=device)

normalized_test_2 = (second_piece[:] - second_piece.mean())/second_piece.std()
test_seq_2, test_labels_2 = create_sequences(normalized_test_2.values, 90)
test_tensor_seq_2 = torch.FloatTensor(test_seq_2).to(device=device)
test_tensor_labels_2 = torch.FloatTensor(test_labels_2).to(device=device)

normalized_test_3 = (third_piece[:] - third_piece.mean())/third_piece.std()
test_seq_3, test_labels_3 = create_sequences(normalized_test_3.values, 90)
test_tensor_seq_3 = torch.FloatTensor(test_seq_3).to(device='cuda')
test_tensor_labels_3 = torch.FloatTensor(test_labels_3).to(device='cuda')

normalized_test_4 = (fourth_piece[:] - fourth_piece.mean())/fourth_piece.std()
test_seq_4, test_labels_4 = create_sequences(normalized_test_4.values, 90)
test_tensor_seq_4 = torch.FloatTensor(test_seq_4).to(device='cuda')
test_tensor_labels_4 = torch.FloatTensor(test_labels_4).to(device='cuda')

test_labels_1_un = test_labels_1 * first_piece.std().values + first_piece.mean().values
test_labels_2_un = test_labels_2 * second_piece.std().values + second_piece.mean().values
test_labels_3_un = test_labels_3 * third_piece.std().values + third_piece.mean().values
test_labels_4_un = test_labels_4 * fourth_piece.std().values + fourth_piece.mean().values

In [22]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, 30)
        # self.fc2 = nn.Linear(30, output_dim)
        # self.do = nn.Dropout()
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)  # Initial hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)  # Initial cell state
        
        out, _ = self.lstm(x, (h0, c0))
        # out = self.do(out)
        # print(out.shape)
        out = self.fc1(out[:,-1, :]) # take the last hidden state because we assume it encodes all the information about the sequence
        # out = self.fc2(out[, :])  # Take the last 30 time step output
        return out.unsqueeze(2)

# Model parameters
input_dim = 1
hidden_dim = 256
num_layers = 3
output_dim = 1

# Instantiate model
model12 = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
model12 = model12.to(device)  # Use "cuda" if you have a GPU
criterion12 = nn.SmoothL1Loss()
optimizer12 = optim.Adam(model12.parameters(), lr=1e-4)

### Train Model 12

In [23]:
num_epochs = 1000
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model12.train()
    
    optimizer12.zero_grad()
    y_pred = model12(train_seq_tensor1)
    
    loss = criterion12(y_pred, train_label_tensor1)
    loss.backward()
    optimizer12.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.47836756706237793
Epoch 10, Train Loss: 0.4753947854042053
Epoch 20, Train Loss: 0.4683681130409241
Epoch 30, Train Loss: 0.43947476148605347
Epoch 40, Train Loss: 0.28645992279052734
Epoch 50, Train Loss: 0.14892318844795227
Epoch 60, Train Loss: 0.13181905448436737
Epoch 70, Train Loss: 0.1069473922252655
Epoch 80, Train Loss: 0.09243323653936386
Epoch 90, Train Loss: 0.08091265708208084
Epoch 100, Train Loss: 0.07327289134263992
Epoch 110, Train Loss: 0.06892047822475433
Epoch 120, Train Loss: 0.06579449772834778
Epoch 130, Train Loss: 0.06309078633785248
Epoch 140, Train Loss: 0.060652755200862885
Epoch 150, Train Loss: 0.05821968987584114
Epoch 160, Train Loss: 0.055515944957733154
Epoch 170, Train Loss: 0.052023012191057205
Epoch 180, Train Loss: 0.047683704644441605
Epoch 190, Train Loss: 0.0440942645072937
Epoch 200, Train Loss: 0.03834317997097969
Epoch 210, Train Loss: 0.035362765192985535
Epoch 220, Train Loss: 0.03199510648846626
Epoch 230, Train Loss

In [24]:
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    model12.train()
    
    optimizer12.zero_grad()
    y_pred = model12(train_seq_tensor2)
    
    loss = criterion12(y_pred, train_label_tensor2)
    loss.backward()
    optimizer12.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.04996645450592041
Epoch 10, Train Loss: 0.025359220802783966
Epoch 20, Train Loss: 0.02131466567516327
Epoch 30, Train Loss: 0.019228655844926834
Epoch 40, Train Loss: 0.020189249888062477
Epoch 50, Train Loss: 0.019007541239261627
Epoch 60, Train Loss: 0.017935512587428093
Epoch 70, Train Loss: 0.01734199747443199
Epoch 80, Train Loss: 0.017354262992739677
Epoch 90, Train Loss: 0.01669156923890114
Epoch 100, Train Loss: 0.016276737675070763
Epoch 110, Train Loss: 0.01598689891397953
Epoch 120, Train Loss: 0.015555349178612232
Epoch 130, Train Loss: 0.015163114294409752
Epoch 140, Train Loss: 0.014603639952838421
Epoch 150, Train Loss: 0.014156438410282135
Epoch 160, Train Loss: 0.013838090933859348
Epoch 170, Train Loss: 0.013275429606437683
Epoch 180, Train Loss: 0.012719825841486454
Epoch 190, Train Loss: 0.012224774807691574
Epoch 200, Train Loss: 0.01118222251534462
Epoch 210, Train Loss: 0.009234367869794369
Epoch 220, Train Loss: 0.010856512002646923
Epoch

In [25]:
torch.save(model12.state_dict, 'model12_30step')

### Avg error model 12 dataset 1

In [26]:
model12.eval()
error = 0
n_sequences = len(test_tensor_seq_1)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_1[i]
    y_test_label = test_labels_1_un[i]
    y_pred_test = model12(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * first_piece.std().values + first_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.006486666238077342


### Avg error model 12 dataset 2

In [27]:
model12.eval()
error = 0
n_sequences = len(test_tensor_seq_2)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_2[i]
    y_test_label = test_labels_2_un[i]
    y_pred_test = model12(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * second_piece.std().values + second_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.0017952547218344499


### Avg error model 12 dataset 3

In [28]:
model12.eval()
error = 0
n_sequences = len(test_tensor_seq_3)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_3[i]
    y_test_label = test_labels_3_un[i]
    y_pred_test = model12(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * third_piece.std().values + third_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.00652512794161521


### Avg error model 12 dataset 4

In [29]:
model12.eval()
error = 0
n_sequences = len(test_tensor_seq_4)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_4[i]
    y_test_label = test_labels_4_un[i]
    y_pred_test = model12(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * fourth_piece.std().values + fourth_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.008781973079321582


### Train model 123

In [30]:
model123 = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
model123 = model123.to(device)  # Use "cuda" if you have a GPU
criterion123 = nn.SmoothL1Loss()
optimizer123 = optim.Adam(model123.parameters(), lr=1e-4)

In [31]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model123.train()
    
    optimizer123.zero_grad()
    y_pred = model123(train_seq_tensor1)
    
    loss = criterion123(y_pred, train_label_tensor1)
    loss.backward()
    optimizer123.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.47917822003364563
Epoch 10, Train Loss: 0.476277619600296
Epoch 20, Train Loss: 0.46947258710861206
Epoch 30, Train Loss: 0.44069620966911316
Epoch 40, Train Loss: 0.2930632531642914
Epoch 50, Train Loss: 0.18702377378940582
Epoch 60, Train Loss: 0.12578678131103516
Epoch 70, Train Loss: 0.11633921414613724
Epoch 80, Train Loss: 0.09690529108047485
Epoch 90, Train Loss: 0.08370163291692734
Epoch 100, Train Loss: 0.07496176660060883
Epoch 110, Train Loss: 0.06966044753789902
Epoch 120, Train Loss: 0.06630214303731918
Epoch 130, Train Loss: 0.06353079527616501
Epoch 140, Train Loss: 0.060938429087400436
Epoch 150, Train Loss: 0.05834558233618736
Epoch 160, Train Loss: 0.055464163422584534
Epoch 170, Train Loss: 0.05168639123439789
Epoch 180, Train Loss: 0.04687570035457611
Epoch 190, Train Loss: 0.04193054884672165
Epoch 200, Train Loss: 0.03688492625951767
Epoch 210, Train Loss: 0.03275888413190842
Epoch 220, Train Loss: 0.02838340774178505
Epoch 230, Train Loss: 

In [32]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model123.train()
    
    optimizer123.zero_grad()
    y_pred = model123(train_seq_tensor2)
    
    loss = criterion123(y_pred, train_label_tensor2)
    loss.backward()
    optimizer123.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.0578572079539299
Epoch 10, Train Loss: 0.029935600236058235
Epoch 20, Train Loss: 0.02190033160150051
Epoch 30, Train Loss: 0.020444955676794052
Epoch 40, Train Loss: 0.019224360585212708
Epoch 50, Train Loss: 0.01844237931072712
Epoch 60, Train Loss: 0.017817240208387375
Epoch 70, Train Loss: 0.017333056777715683
Epoch 80, Train Loss: 0.016913514584302902
Epoch 90, Train Loss: 0.016534360125660896
Epoch 100, Train Loss: 0.01616121269762516
Epoch 110, Train Loss: 0.01577279344201088
Epoch 120, Train Loss: 0.015351208858191967
Epoch 130, Train Loss: 0.014847041107714176
Epoch 140, Train Loss: 0.014075756072998047
Epoch 150, Train Loss: 0.011811651289463043
Epoch 160, Train Loss: 0.01129104197025299
Epoch 170, Train Loss: 0.013670951128005981
Epoch 180, Train Loss: 0.013289576396346092
Epoch 190, Train Loss: 0.011477327905595303
Epoch 200, Train Loss: 0.010716233402490616
Epoch 210, Train Loss: 0.01044456847012043
Epoch 220, Train Loss: 0.015613737516105175
Epoch 2

In [33]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model123.train()
    
    optimizer123.zero_grad()
    y_pred = model123(train_seq_tensor3)
    
    loss = criterion123(y_pred, train_label_tensor3)
    loss.backward()
    optimizer123.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.11098272353410721
Epoch 10, Train Loss: 0.04813872277736664
Epoch 20, Train Loss: 0.030873211100697517
Epoch 30, Train Loss: 0.025025205686688423
Epoch 40, Train Loss: 0.021152237430214882
Epoch 50, Train Loss: 0.020658152177929878
Epoch 60, Train Loss: 0.020544126629829407
Epoch 70, Train Loss: 0.020221494138240814
Epoch 80, Train Loss: 0.0170571431517601
Epoch 90, Train Loss: 0.024117929860949516
Epoch 100, Train Loss: 0.021127047017216682
Epoch 110, Train Loss: 0.01582333631813526
Epoch 120, Train Loss: 0.013477539643645287
Epoch 130, Train Loss: 0.012381318025290966
Epoch 140, Train Loss: 0.023574350401759148
Epoch 150, Train Loss: 0.020463744178414345
Epoch 160, Train Loss: 0.018213149160146713
Epoch 170, Train Loss: 0.016685089096426964
Epoch 180, Train Loss: 0.015592044219374657
Epoch 190, Train Loss: 0.015022439882159233
Epoch 200, Train Loss: 0.014589119702577591
Epoch 210, Train Loss: 0.014246814884245396
Epoch 220, Train Loss: 0.013963752426207066
Epoc

In [35]:
torch.save(model123.state_dict, 'model123_30step')

### Avg error model 123 dataset 1

In [36]:
model123.eval()
error = 0
n_sequences = len(test_tensor_seq_1)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_1[i]
    y_test_label = test_labels_1_un[i]
    y_pred_test = model123(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * first_piece.std().values + first_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.0081342174702184


### Avg error model 123 dataset 2

In [37]:
model123.eval()
error = 0
n_sequences = len(test_tensor_seq_2)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_2[i]
    y_test_label = test_labels_2_un[i]
    y_pred_test = model123(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * second_piece.std().values + second_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.007515270791341105


### Avg error model 123 dataset 3

In [38]:
model123.eval()
error = 0
n_sequences = len(test_tensor_seq_3)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_3[i]
    y_test_label = test_labels_3_un[i]
    y_pred_test = model123(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * third_piece.std().values + third_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.0016922676836385467


### Avg error model 123 dataset 4

In [39]:
model123.eval()
error = 0
n_sequences = len(test_tensor_seq_4)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_4[i]
    y_test_label = test_labels_4_un[i]
    y_pred_test = model123(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * fourth_piece.std().values + fourth_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.008531851870275827


### Train model 124

In [40]:
model124 = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
model124 = model124.to(device)  # Use "cuda" if you have a GPU
criterion124 = nn.SmoothL1Loss()
optimizer124 = optim.Adam(model124.parameters(), lr=1e-4)

In [41]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model124.train()
    
    optimizer124.zero_grad()
    y_pred = model124(train_seq_tensor1)
    
    loss = criterion124(y_pred, train_label_tensor1)
    loss.backward()
    optimizer124.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.4786578118801117
Epoch 10, Train Loss: 0.4757179915904999
Epoch 20, Train Loss: 0.46889129281044006
Epoch 30, Train Loss: 0.4396200478076935
Epoch 40, Train Loss: 0.28651735186576843
Epoch 50, Train Loss: 0.18400731682777405
Epoch 60, Train Loss: 0.1330108493566513
Epoch 70, Train Loss: 0.11879938840866089
Epoch 80, Train Loss: 0.09723170101642609
Epoch 90, Train Loss: 0.08358990401029587
Epoch 100, Train Loss: 0.07411117851734161
Epoch 110, Train Loss: 0.06837597489356995
Epoch 120, Train Loss: 0.06493419408798218
Epoch 130, Train Loss: 0.0621645525097847
Epoch 140, Train Loss: 0.05963553860783577
Epoch 150, Train Loss: 0.05717995762825012
Epoch 160, Train Loss: 0.054595377296209335
Epoch 170, Train Loss: 0.05159774050116539
Epoch 180, Train Loss: 0.04755287989974022
Epoch 190, Train Loss: 0.045041751116514206
Epoch 200, Train Loss: 0.042288996279239655
Epoch 210, Train Loss: 0.03778833523392677
Epoch 220, Train Loss: 0.03335537761449814
Epoch 230, Train Loss: 0

In [42]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model124.train()
    
    optimizer124.zero_grad()
    y_pred = model124(train_seq_tensor2)
    
    loss = criterion124(y_pred, train_label_tensor2)
    loss.backward()
    optimizer124.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.05277636647224426
Epoch 10, Train Loss: 0.025545308366417885
Epoch 20, Train Loss: 0.023558514192700386
Epoch 30, Train Loss: 0.02070009708404541
Epoch 40, Train Loss: 0.01965363882482052
Epoch 50, Train Loss: 0.018774697557091713
Epoch 60, Train Loss: 0.01826183684170246
Epoch 70, Train Loss: 0.01784948632121086
Epoch 80, Train Loss: 0.01750996522605419
Epoch 90, Train Loss: 0.017197223380208015
Epoch 100, Train Loss: 0.016890998929739
Epoch 110, Train Loss: 0.016598952934145927
Epoch 120, Train Loss: 0.01630241610109806
Epoch 130, Train Loss: 0.016686322167515755
Epoch 140, Train Loss: 0.0159809160977602
Epoch 150, Train Loss: 0.015231838449835777
Epoch 160, Train Loss: 0.014716466888785362
Epoch 170, Train Loss: 0.013975403271615505
Epoch 180, Train Loss: 0.01320638321340084
Epoch 190, Train Loss: 0.012558561749756336
Epoch 200, Train Loss: 0.01150307897478342
Epoch 210, Train Loss: 0.009287677705287933
Epoch 220, Train Loss: 0.008533104322850704
Epoch 230, Tr

In [43]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model124.train()
    
    optimizer124.zero_grad()
    y_pred = model124(train_seq_tensor4)
    
    loss = criterion124(y_pred, train_label_tensor4)
    loss.backward()
    optimizer124.step()
    
    train_losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {loss.item()}')

Epoch 0, Train Loss: 0.11766194552183151
Epoch 10, Train Loss: 0.04128238186240196
Epoch 20, Train Loss: 0.027846086770296097
Epoch 30, Train Loss: 0.023181550204753876
Epoch 40, Train Loss: 0.02023240737617016
Epoch 50, Train Loss: 0.01822824962437153
Epoch 60, Train Loss: 0.016674017533659935
Epoch 70, Train Loss: 0.015018023550510406
Epoch 80, Train Loss: 0.013286682777106762
Epoch 90, Train Loss: 0.01113374624401331
Epoch 100, Train Loss: 0.008656430058181286
Epoch 110, Train Loss: 0.00716521218419075
Epoch 120, Train Loss: 0.0064131394028663635
Epoch 130, Train Loss: 0.005959879141300917
Epoch 140, Train Loss: 0.005641344469040632
Epoch 150, Train Loss: 0.005377114284783602
Epoch 160, Train Loss: 0.005170464515686035
Epoch 170, Train Loss: 0.004951815586537123
Epoch 180, Train Loss: 0.004777141381055117
Epoch 190, Train Loss: 0.004596447106450796
Epoch 200, Train Loss: 0.004416464362293482
Epoch 210, Train Loss: 0.004221882671117783
Epoch 220, Train Loss: 0.004331293981522322
Epoc

In [44]:
torch.save(model124.state_dict, 'model124_30step')

### Avg error model 124 dataset 1

In [45]:
model124.eval()
error = 0
n_sequences = len(test_tensor_seq_1)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_1[i]
    y_test_label = test_labels_1_un[i]
    y_pred_test = model124(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * first_piece.std().values + first_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.009822727435360345


### Avg error model 124 dataset 2

In [46]:
model124.eval()
error = 0
n_sequences = len(test_tensor_seq_2)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_2[i]
    y_test_label = test_labels_2_un[i]
    y_pred_test = model124(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * second_piece.std().values + second_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.008765140381459976


### Avg error model 124 datset 3

In [47]:
model124.eval()
error = 0
n_sequences = len(test_tensor_seq_3)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_3[i]
    y_test_label = test_labels_3_un[i]
    y_pred_test = model124(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * third_piece.std().values + third_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.007502452232763586


### Avg error model 124 dataset 4

In [48]:
model124.eval()
error = 0
n_sequences = len(test_tensor_seq_4)
for i in range(n_sequences):
    # ignore the last step since we don't have real data to compare with
    y_test_seq = test_tensor_seq_4[i]
    y_test_label = test_labels_4_un[i]
    y_pred_test = model124(y_test_seq.unsqueeze(0))
    # un-normalize prediction and label
    y_pred_test = y_pred_test.squeeze(0).detach().cpu().numpy() * fourth_piece.std().values + fourth_piece.mean().values
    mare = MARE_np(y_pred_test, y_test_label).item()
    error += mare

avg_error = error/n_sequences
print(f'AVG error across sequences: {avg_error}')

AVG error across sequences: 0.0009080997452065353
